__BIOBSS - Polar reader__

_This notebook includes guidelines to save Polar data files as csv file and and synchronize the signal from different sensors._

In [1]:
#Import BIOBSS and the other required packages

import biobss
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

#help(biobss)

# Table of Contents
1.[Polar Sample Data](#polar)<br>
2.[Timestamp Conversion](#polar_time)<br>
3.[Synchronization of Signals from Different Sensors](#polar_sync)<br>
4.[Data Segmentation Based on Events](#polar_event)<br>
5.[Loading and Saving Alternatives](#polar_files)<br>

### __Polar Sample Data__
<a id="polar"></a>

In [ ]:
#Define the file paths for the files
txt_dir="C:\\Users\\ipek.karakus\\Desktop\\biobss\\Polar_files\\txt_files\\"
csv_dir="C:\\Users\\ipek.karakus\\Desktop\\biobss\\Polar_files\\csv_files\\"
pkl_dir="C:\\Users\\ipek.karakus\\Desktop\\biobss\\Polar_files\\pkl_files\\"

#Txt files can be saved as csv files
biobss.reader.polar_format.txt_to_csv(txt_dir)

### __Timestamp Conversion__
<a id="polar_time"></a>

In [ ]:
#Timestamps in iso format can be converted to milliseconds for a specific file. This can be done by referencing either to start time of the sensor or a given start time.
filepath=r'C:\Users\ipek.karakus\Desktop\biobss\Polar_files\csv_files\ik_d1_25042022_forearm_left\Polar_Sense_A40F6F28_20220425_113903_PPG.csv'
df= pd.read_csv(filepath)
time_msec=biobss.reader.polar_format.timestamp_to_msec(df['Phone timestamp']) 

#If required, the selected csv files can be updated by adding a 'Time_record (ms)' column, corresponds to time points in ms. The timepoints are calculated referenced to the earliest timestamp for all sensors.
csv_subdir="C:\\Users\\ipek.karakus\\Desktop\\biobss\\Polar_files\\csv_files\\ik_d1_25042022_forearm_left" #This time, the path should be defined for a specific record.
biobss.reader.update_csv(csv_dir=csv_subdir,marker=True)

### __Synchronization of Signals from Different Sensors__
<a id="polar_sync"></a>

In [ ]:
#For synchronization of the signals, a common (overlapping region for all sensors) time array can be generated. This time, the path should be defined for a specific record.
time_list=biobss.reader.calculate_sync_time(csv_dir=csv_subdir,time_step=1,marker=True)

#If required, the time array can be saved as a txt file.
time_list=biobss.reader.calculate_sync_time(csv_dir=csv_subdir,time_step=1,marker=True,save_file=True)

#The signals can be synchronized by interpolating the signal for the given time_list.
data=biobss.reader.synchronize_signals(csv_subdir,time_list)

#Resampling can be applied if the required parameters (sampling_rate and resampling_rate are given) 
data=biobss.reader.synchronize_signals(csv_subdir,time_list,resampling_rate=100)

#If required, the synchronized signals can be saved as a csv file.
data=biobss.reader.synchronize_signals(csv_subdir,time_list,resampling_rate=100,save_files=True)


### __Data Segmentation Based on Events__
<a id="polar_event"></a>

In [ ]:
#The synchronized signals can be segmented for events
filepath=r'C:\Users\ipek.karakus\Desktop\biobss\Polar_files\csv_files\ik_d1_25042022_forearm_left\sync_ACC_PPG_MAGN_GYRO.csv'
markerpath=r'C:\Users\ipek.karakus\Desktop\biobss\Polar_files\csv_files\ik_d1_25042022_forearm_left\MARKER_20220425_114002.csv'
out_path=r'C:\Users\ipek.karakus\Desktop\biobss\Polar_files\csv_files\ik_d1_25042022_forearm_left\event_list.txt'
data=biobss.reader.segment_events(filepath,markerpath,['rest'],out_path)

#The event list can also be saved as txt file for later use
biobss.reader.segment_events(filepath,markerpath,['rest'],out_path,save_file=True)



### __Loading and Saving Alternatives__
<a id="polar_files"></a>

In [ ]:
#Csv file can be read into a dictionary
filepath=r'C:\Users\ipek.karakus\Desktop\biobss\Polar_files\csv_files\ik_d1_25042022_forearm_left\Polar_Sense_A40F6F28_20220425_113903_PPG.csv'
data=biobss.reader.polar_csv_reader(filepath,signal_type='PPG')

#Csv files can be saved as pkl files
biobss.reader.csv_to_pkl(csv_dir)